# Objective 2: Demand Forecasting by Branch

Forecasts monthly demand per branch using linear regression (with Prophet as optional upgrade).

In [ ]:
import sys; sys.path.insert(0, '..')
import pandas as pd
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')

from src.data.ingestion import load_monthly_sales
from src.models.demand_forecaster import forecast_all_branches

monthly = load_monthly_sales()
print(monthly)

In [ ]:
# Plot historical monthly sales by branch
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for i, branch in enumerate(monthly['branch'].unique()):
    bdf = monthly[monthly['branch'] == branch]
    ax = axes[i]
    ax.plot(bdf['month_num'].astype(str) + '/' + bdf['year'].astype(str),
            bdf['total_sales'], marker='o')
    ax.set_title(branch)
    ax.set_xlabel('Period')
    ax.set_ylabel('Sales')
    ax.tick_params(axis='x', rotation=45)

plt.suptitle('Monthly Sales by Branch (Historical)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Run forecasts
results = forecast_all_branches(monthly, n_months=3)
print('\nDemand Ranking:')
for r in results['demand_ranking']:
    print(f"  {r['branch']:25s} → avg forecast {r['avg_forecast']:>15,.0f}")

In [ ]:
# Print branch insights
for branch, bdata in results['forecasts'].items():
    if 'error' not in bdata:
        print(f'\n{branch}:')
        print(f'  Growth: {bdata["growth_pct_over_period"]:+.1f}%')
        print(f'  Insight: {bdata["insight"]}')
        for f in bdata['forecast']:
            print(f'    {f["period"]}: {f["forecast"]:,.0f} [{f["lower"]:,.0f} – {f["upper"]:,.0f}]')